In [1]:
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True,
   text=True)
print(result.stdout)

Sun Oct  5 13:27:39 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:08.0 Off |                  Off |
| N/A   28C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
 !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 7.7 MB/s  0:02:456m0:00:0100:05
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.14.1+cu116 (from versions: 0.1.6, 0.2.0, 0.15.0+cpu, 0.15.0+cu117, 0.15.0+cu118, 0.15.1, 0.15.1+cpu, 0.15.1+cu117, 0.15.1+cu118, 0.15.2, 0.15.2+cpu, 0.15.2+cu117, 0.15.2+cu118, 0.15.2+rocm5.3, 0.15.2+rocm5.4.2, 0.16.0, 0.16.0+cpu, 0.16.0+cu118, 0.16.0+cu121, 0.16.0+rocm5.5, 0.16.0+rocm5.6, 0.16.1, 0.16.1+cpu, 0.16.1+cu118, 0.16.1+cu121, 0.16.1+rocm5.5, 0.16.1+rocm5.6, 0.16.2, 0.16.2+cpu, 0.16.2+cu118, 0.16.2+cu121, 0.16.2+rocm5.5, 0.16.2+rocm5.6, 0.17.0, 0.17.0+cpu, 0.17.0+cu118, 0.17.0+cu121, 0.17.0+rocm5.6, 0.17.0+rocm5.7, 0.17.1, 0.17.1+cpu, 0.17.1+cu118, 0.17

In [3]:
import torch
print(f"新PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU测试: {torch.cuda.get_device_name(0)}")
    # 简单GPU测试
    x = torch.randn(100, 100).cuda()
    y = torch.mm(x, x.t())
    print("✅ GPU正常工作!")
import torch

新PyTorch版本: 2.3.1+cu121
CUDA可用: True
GPU测试: Tesla P100-PCIE-16GB
✅ GPU正常工作!


In [3]:
"""
将BERT集成到您现有的训练代码中 - ModelScope版本
使用ModelScope BERT模型进行多标签分类
"""

import torch
import torch.nn as nn
from torch.optim import AdamW
from modelscope import AutoModel, AutoTokenizer
import pandas as pd
import os
import numpy
print(f"PyTorch版本: {torch.__version__}")
print(f"NumPy: {numpy.__version__}")


# 工具类
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
class Timer:
    def __init__(self):
        import time
        self.time = time
        self.start_time = self.time.time()
    def stop(self):
        return self.time.time() - self.start_time

def try_all_gpus():
    """检测可用GPU"""
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ============ ModelScope BERT模型 ============
# 模型定义
class BERTSentimentClassifier(nn.Module):
    def __init__(self, model_name='AI-ModelScope/distilbert-base-uncased', num_classes=6, dropout=0.1):
        super(BERTSentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        print(f"✅ 加载模型: {model_name}")
        # 冻结策略
        for param in self.bert.embeddings.parameters():
            param.requires_grad = False
        if hasattr(self.bert, 'transformer'):
            for layer in self.bert.transformer.layer[:-1]:
                for param in layer.parameters():
                    param.requires_grad = False
        self.dropout = nn.Dropout(dropout)
        hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_classes)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        return self.classifier(self.dropout(pooled_output))

# ============ 修改后的训练函数 ============
def train_bert_epoch(net, train_iter, loss, updater, device):
    """
    单个epoch训练 - 添加混合精度训练
    """
    net.train()
    metric = Accumulator(3)  # 训练损失总和, 准确数, 样本数

    # 使用混合精度训练
    scaler = torch.cuda.amp.GradScaler() if device.type == 'cuda' else None

    for _, batch in enumerate(train_iter):
        # 解析batch
        if len(batch) == 3:
            input_ids, attention_mask, labels = batch
        else:
            input_ids, attention_mask, labels = batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 混合精度前向传播
        if scaler is not None:
            with torch.cuda.amp.autocast():
                y_hat = net(input_ids, attention_mask)
                l = loss(y_hat, labels)
        else:
            y_hat = net(input_ids, attention_mask)
            l = loss(y_hat, labels)

        # 反向传播
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            if scaler is not None:
                scaler.scale(l.sum()).backward()
                scaler.unscale_(updater)
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
                scaler.step(updater)
                scaler.update()
            else:
                l.sum().backward()
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
                updater.step()

        # 统计
        with torch.no_grad():
            acc = multilabel_accuracy(y_hat, labels)
            metric.add(l.sum(), acc * labels.shape[0], labels.shape[0])

    return metric[0] / metric[2], metric[1] / metric[2]


# 训练函数
def multilabel_accuracy(y_hat, y):
    predictions = torch.sigmoid(y_hat) > 0.5
    y = y.bool()
    label_wise_acc = (predictions == y).float().mean()
    return label_wise_acc.item()
def train_bert_epoch(net, train_iter, loss, updater, device):
    net.train()
    metric = Accumulator(3)
    for _, batch in enumerate(train_iter):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        y_hat = net(input_ids, attention_mask)
        l = loss(y_hat, labels)
        updater.zero_grad()
        l.sum().backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(),max_norm=1.0)
        updater.step()
        with torch.no_grad():
            acc = multilabel_accuracy(y_hat, labels)
            metric.add(l.sum(), acc * labels.shape[0],labels.shape[0])
    return metric[0] / metric[2], metric[1] / metric[2]

def evaluate_bert_accuracy(net, data_iter, device):
    net.eval()
    metric = Accumulator(2)
    with torch.no_grad():
        for batch in data_iter:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            y_hat = net(input_ids, attention_mask)
            acc = multilabel_accuracy(y_hat, labels)
            metric.add(acc * labels.shape[0], labels.shape[0])
    return metric[0] / metric[1]

def train_bert_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices, scheduler=None):
    """
    完整训练流程
    """
    print('training on', devices)

    if isinstance(devices, list) and len(devices) > 1:
        # 多GPU
        net = nn.DataParallel(net, device_ids=devices)

    device = devices[0] if isinstance(devices, list) else devices
    net = net.to(device)

    timer = Timer()

    for epoch in range(num_epochs):
        # 训练
        train_loss, train_acc = train_bert_epoch(
            net, train_iter, loss, trainer, device
        )

        # 验证
        test_acc = evaluate_bert_accuracy(net, test_iter, device)

        # 学习率调度
        if scheduler is not None:
            scheduler.step()

        print(f'Epoch {epoch + 1}: '
              f'loss {train_loss:.3f}, '
              f'train acc {train_acc:.3f}, '
              f'test acc {test_acc:.3f}')

    print(f'Training completed in {timer.stop():.1f} sec')
    print(f'Final: train acc {train_acc:.3f}, test acc {test_acc:.3f}')


def read_toxic_comments_real(data_dir, max_samples=None, is_train=True):
    """
    读取真实的Kaggle Toxic Comment Classification数据
    返回格式: (texts, labels, ids)
    """
    import pandas as pd
    import os

    if is_train:
        csv_path = os.path.join(data_dir, 'train.csv')
        print(f"读取训练数据: {csv_path}")

        df = pd.read_csv(csv_path)
        if max_samples:
            df = df.head(max_samples)

        texts = df['comment_text'].tolist()
        label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        labels = df[label_columns].values.tolist()
        ids = df['id'].tolist()

        print(f"加载训练数据: {len(texts)} 条")
        print(f"标签分布: {dict(zip(label_columns, df[label_columns].sum().tolist()))}")

        return texts, labels, ids
    else:
        csv_path = os.path.join(data_dir, 'test.csv')
        print(f"读取测试数据: {csv_path}")

        df = pd.read_csv(csv_path)
        if max_samples:
            df = df.head(max_samples)

        texts = df['comment_text'].tolist()
        ids = df['id'].tolist()

        print(f"加载测试数据: {len(texts)} 条")

        return texts, None, ids

def generate_submission(model, test_loader, device, test_ids, output_path):
    """
    生成Kaggle提交文件
    """
    import pandas as pd

    model.eval()
    predictions = []

    print("🔮 生成预测结果...")
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, _ = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # 获取logits并转换为概率
            logits = model(input_ids, attention_mask)
            probs = torch.sigmoid(logits).cpu().numpy()
            predictions.extend(probs)

    # 创建提交DataFrame
    label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    submission_df = pd.DataFrame({
        'id': test_ids,
        **{col: [pred[i] for pred in predictions] for i, col in enumerate(label_columns)}
    })

    # 保存提交文件
    submission_df.to_csv(output_path, index=False)
    print(f"💾 提交文件已保存: {output_path}")
    print(f"📊 预测统计:")
    for i, col in enumerate(label_columns):
        avg_prob = sum(pred[i] for pred in predictions) / len(predictions)
        print(f"  {col}: 平均概率 {avg_prob:.4f}")

    return submission_df

# 数据集
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.float32)
        else:
            self.labels = torch.zeros((len(texts), 6),
dtype=torch.float32)
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return (
            encoding['input_ids'].squeeze(),
            encoding['attention_mask'].squeeze(),
            self.labels[idx]
        )
    def __len__(self):
        return len(self.labels)

PyTorch版本: 2.8.0+cu128
NumPy: 1.26.4


In [3]:
  import pandas as pd
  import os

  print("🔄 测试数据加载...")
  data_dir = 'toxic-comment'

  try:
      # 只读取几行数据
      train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'),
  nrows=10)
      print(f"✅ 训练数据读取成功: {len(train_df)} 行")

      test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'),
  nrows=10)
      print(f"✅ 测试数据读取成功: {len(test_df)} 行")

      print("数据加载完成，没有问题")
  except Exception as e:
      print(f"❌ 数据加载失败: {e}")


🔄 测试数据加载...
✅ 训练数据读取成功: 10 行
✅ 测试数据读取成功: 10 行
数据加载完成，没有问题


In [13]:
try:
    # 先尝试简单的tokenizer
    from modelscope import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained('AI-ModelScope/bert-base-uncased')
    # 测试编码
    text = "This is a test"
    encoding = tokenizer(text, max_length=32, truncation=True,
padding='max_length', return_tensors='pt')
    print(f"✅ Tokenizer测试成功")
    print(f"编码shape: {encoding['input_ids'].shape}")
except Exception as e:
    print(f"❌ Tokenizer测试失败: {e}")

✅ Tokenizer测试成功
编码shape: torch.Size([1, 32])


In [14]:
import torch
import torch.nn as nn
try:
    print("🔄 测试基础BERT模型加载...")
    from modelscope import AutoModel
    # 使用最简单的BERT模型
    model = AutoModel.from_pretrained('AI-ModelScope/distilbert-base-uncased')
    print(f"✅ BERT模型加载成功")
    # 测试前向传播
    input_ids = torch.randint(0, 1000, (1, 32))
    attention_mask = torch.ones(1, 32)
    with torch.no_grad():
        outputs = model(input_ids=input_ids,
attention_mask=attention_mask)
    print(f"✅ 模型前向传播成功: {outputs.last_hidden_state.shape}")
except Exception as e:
    print(f"❌ 模型测试失败: {e}")
    import traceback
    traceback.print_exc()

🔄 测试基础BERT模型加载...


2025-10-04 21:09:24.727865: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-04 21:09:26.352233: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ BERT模型加载成功
✅ 模型前向传播成功: torch.Size([1, 32, 768])


In [16]:
try:
    print("🔄 测试完整分类模型创建...")
    class BERTSentimentClassifier(nn.Module):
        def __init__(self, 
model_name='AI-ModelScope/distilbert-base-uncased', num_classes=6, 
dropout=0.1):
            super(BERTSentimentClassifier, self).__init__()
            from modelscope import AutoModel
            self.bert = AutoModel.from_pretrained(model_name)
            print(f"✅ 加载模型: {model_name}")
            # 冻结策略 - 只训练最后1层和分类器
            for param in self.bert.embeddings.parameters():
                param.requires_grad = False
            # DistilBERT结构
            if hasattr(self.bert, 'transformer'):
                for layer in self.bert.transformer.layer[:-1]:
                    for param in layer.parameters():
                        param.requires_grad = False
            self.dropout = nn.Dropout(dropout)
            hidden_size = self.bert.config.hidden_size
            self.classifier = nn.Linear(hidden_size, num_classes)
        def forward(self, input_ids, attention_mask):
            outputs = self.bert(input_ids=input_ids,
attention_mask=attention_mask)
            pooled_output = outputs.last_hidden_state[:, 0]
            return self.classifier(self.dropout(pooled_output))
    # 创建模型
    net = BERTSentimentClassifier(num_classes=6, dropout=0.1)
    print(f"✅ 分类模型创建成功")
    # 测试前向传播
    input_ids = torch.randint(0, 1000, (2, 32))
    attention_mask = torch.ones(2, 32)
    with torch.no_grad():
        logits = net(input_ids, attention_mask)
    print(f"✅ 分类模型前向传播成功: {logits.shape}")
    print(f"📊 输出logits范围: {logits.min():.3f} ~ {logits.max():.3f}")
    # 测试sigmoid概率
    probs = torch.sigmoid(logits)
    print(f"📊 sigmoid概率范围: {probs.min():.3f} ~ {probs.max():.3f}")
except Exception as e:
    print(f"❌ 模型创建失败: {e}")
    import traceback
    traceback.print_exc()

🔄 测试完整分类模型创建...
✅ 加载模型: AI-ModelScope/distilbert-base-uncased
✅ 分类模型创建成功
✅ 分类模型前向传播成功: torch.Size([2, 6])
📊 输出logits范围: -0.309 ~ 0.298
📊 sigmoid概率范围: 0.423 ~ 0.574


In [17]:
try:
    print("🔄 测试训练循环组件...")
    from torch.optim import AdamW
    # 测试损失函数
    loss_fn = nn.BCEWithLogitsLoss(reduction="none")
    # 创建模拟数据
    batch_size = 2
    num_classes = 6
    logits = torch.randn(batch_size, num_classes)
    labels = torch.randint(0, 2, (batch_size, num_classes)).float()
    print(f"📊 模拟数据:")
    print(f"  logits shape: {logits.shape}")
    print(f"  labels shape: {labels.shape}")
    print(f"  labels 示例: {labels[0]}")
    # 测试损失计算
    loss = loss_fn(logits, labels)
    print(f"✅ 损失计算成功: {loss.shape}, 平均损失: {loss.mean():.4f}")
    # 测试准确率计算
    def multilabel_accuracy(y_hat, y):
        predictions = torch.sigmoid(y_hat) > 0.5
        y = y.bool()
        label_wise_acc = (predictions == y).float().mean()
        return label_wise_acc.item()
    acc = multilabel_accuracy(logits, labels)
    print(f"✅ 准确率计算成功: {acc:.4f}")
    # 测试优化器
    optimizer = AdamW([
        {'params': net.classifier.parameters(), 'lr': 1e-3},
        {'params': [p for p in net.bert.parameters() if p.requires_grad], 'lr': 1e-4}
    ], weight_decay=0.01)
    print(f"✅ 优化器创建成功")
    # 测试一步训练
    optimizer.zero_grad()
    logits = net(input_ids, attention_mask)
    loss = loss_fn(logits, labels).mean()
    loss.backward()
    optimizer.step()
    print(f"✅ 训练步骤成功, 损失: {loss.item():.4f}")
except Exception as e:
    print(f"❌ 训练循环测试失败: {e}")
    import traceback
    traceback.print_exc()

🔄 测试训练循环组件...
📊 模拟数据:
  logits shape: torch.Size([2, 6])
  labels shape: torch.Size([2, 6])
  labels 示例: tensor([0., 0., 0., 0., 0., 0.])
✅ 损失计算成功: torch.Size([2, 6]), 平均损失: 0.9190
✅ 准确率计算成功: 0.3333
✅ 优化器创建成功
✅ 训练步骤成功, 损失: 0.6747


In [ ]:
# ============ 主要执行代码 ============
print("🚀 启动ModelScope BERT多标签分类训练")
# 数据目录
data_dir = 'toxic-comment'
# 数据加载
print("📊 加载真实Kaggle数据...")
# 为了快速训练，限制样本数（可以根据需要调整）
train_texts, train_labels, train_ids = read_toxic_comments_real(
    data_dir, max_samples=1000, is_train=True
)
# 创建验证集（从训练数据中分割）
val_split = int(len(train_texts) * 0.8)
val_texts = train_texts[val_split:]
val_labels = train_labels[val_split:]
train_texts = train_texts[:val_split]
train_labels = train_labels[:val_split]
# 读取测试数据
test_texts, _, test_ids = read_toxic_comments_real(
    data_dir, max_samples=1000, is_train=False
)
# 初始化tokenizer - 使用指定的ModelScope BERT模型
tokenizer = AutoTokenizer.from_pretrained('AI-ModelScope/bert-base-uncased')
print(f"\n📊 数据统计:")
print(f"训练数据: {len(train_texts)} 条")
print(f"验证数据: {len(val_texts)} 条")
print(f"测试数据: {len(test_texts)} 条")
# 检查数据质量
print(f"\n📝 数据样例:")
print(f"文本长度: {len(train_texts[0])}")
print(f"前100字符: {train_texts[0][:100]}")
print(f"标签: {train_labels[0]}")
# 检查tokenizer
sample_encoding = tokenizer(train_texts[0], max_length=128, truncation=True, padding='max_length', return_tensors='pt')
print(f"\n🔧 Tokenizer测试:")
print(f"input_ids shape: {sample_encoding['input_ids'].shape}")
print(f"attention_mask shape: {sample_encoding['attention_mask'].shape}")
print(f"实际token数: {sample_encoding['attention_mask'].sum()}")
# 模型参数
num_classes = 6  # 6个类别：toxic, severe_toxic, obscene, threat, insult, identity_hate
dropout = 0.1
batch_size = 16  # 适配在线环境
num_steps = 32  # 序列长度
lr = 1e-4  # 学习率
num_epochs = 3  # 增加到3个epoch获得更好效果
 # 使用ModelScope上实际存在的BERT模型
model_name = 'AI-ModelScope/distilbert-base-uncased'
# 创建模型
net = BERTSentimentClassifier(
    model_name=model_name,
    num_classes=num_classes,
    dropout=dropout
)

# 创建数据加载器
train_dataset = BERTDataset(train_texts, train_labels, tokenizer, max_length=num_steps)
val_dataset = BERTDataset(val_texts, val_labels, tokenizer, max_length=num_steps)
test_dataset = BERTDataset(test_texts, None, tokenizer, max_length=num_steps)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                         num_workers=0, pin_memory=False)
val_iter = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                                       num_workers=0, pin_memory=False)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                        num_workers=0, pin_memory=False)
# 优化器 - 只训练最后一层和分类器
trainer = AdamW([
    {'params': [p for name, p in net.named_parameters() if 'classifier' in name], 'lr': lr * 10},  # 分类层
    {'params': [p for name, p in net.named_parameters() if 'classifier' not in name and p.requires_grad], 'lr': lr}  # BERT层
], weight_decay=0.01)
# 损失函数 - 多标签分类使用BCEWithLogitsLoss
loss = nn.BCEWithLogitsLoss(reduction="none")  # 每个样本每个标签独立计算
# 训练
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 使用设备: {device}")
# 开始训练
train_bert_ch13(net, train_iter, val_iter, loss, trainer, num_epochs, device, None)
# 生成提交文件
print("\n" + "="*60)
print("📤 生成Kaggle提交文件...")
print("="*60)
submission_path = 'toxic_comment/submission.csv'
submission_df = generate_submission(net, test_iter, device, test_ids, submission_path)
print("\n" + "="*60)
print("🎉 ModelScope BERT训练和预测完成!")
print("="*60)
print(f"✅ 模型:{model_name}")
print(f"✅ 训练样本: {len(train_texts)}")
print(f"✅ 验证样本: {len(val_texts)}")
print(f"✅ 测试样本: {len(test_texts)}")
print(f"✅ 提交文件: {submission_path}")
print("✅ 支持多标签分类")
print("✅ 混合精度训练")
print("="*60)

🚀 启动ModelScope BERT多标签分类训练
📊 加载真实Kaggle数据...
读取训练数据: toxic-comment/train.csv
加载训练数据: 1000 条
标签分布: {'toxic': 105, 'severe_toxic': 9, 'obscene': 49, 'threat': 4, 'insult': 54, 'identity_hate': 9}
读取测试数据: toxic-comment/test.csv
加载测试数据: 1000 条

📊 数据统计:
训练数据: 800 条
验证数据: 200 条
测试数据: 1000 条

📝 数据样例:
文本长度: 264
前100字符: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't 
标签: [0, 0, 0, 0, 0, 0]

🔧 Tokenizer测试:
input_ids shape: torch.Size([1, 128])
attention_mask shape: torch.Size([1, 128])
实际token数: 68


2025-10-04 21:32:58.257984: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-04 21:32:59.915545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ 加载模型: AI-ModelScope/distilbert-base-uncased
🔥 使用设备: cuda
training on cuda
